# Darts

In [1]:
from keras.models import Sequential
from keras.layers import Dense

In [2]:
# Instantiate a sequential model
model = Sequential()

In [3]:
# Add 3 dense layers of 128, 64 and 32 neurons each
model.add(Dense(128, input_shape=(2,), activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))

In [4]:
# Add a dense layer with as many neurons as competitors
model.add(Dense(4, activation='softmax'))

In [5]:
# Compile your model using categorical_crossentropy loss
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

## Prepare your dataset

In [6]:
import pandas as pd

In [7]:
darts = pd.read_csv('darts.csv')

In [8]:
# Transform into a categorical variable
darts.competitor = pd.Categorical(darts.competitor)

In [9]:
# Assign a number to each category (label encoding)
darts.competitor = darts.competitor.cat.codes

In [10]:
# Print the label encoded competitors
print('Label encoded competitors: \n',darts.competitor.head())

Label encoded competitors: 
 0    2
1    3
2    1
3    0
4    2
Name: competitor, dtype: int8


In [11]:
from keras.utils import to_categorical

In [12]:
coordinates = darts.drop(['competitor'], axis=1)
# Use to_categorical on your labels
competitors = to_categorical(darts.competitor)

In [13]:
print('One-hot encoded competitors: \n',competitors)

One-hot encoded competitors: 
 [[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 ...
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]


## Training on dart throwers

In [14]:
from sklearn.model_selection import train_test_split

In [19]:
coord_train, coord_test, competitors_train, competitors_test = train_test_split(coordinates, competitors, test_size=0.25, stratify=competitors)

In [20]:
# Fit your model to the training data for 200 epochs
model.fit(coord_train, competitors_train,epochs=200)

och 2/200
19/19 [==============================] - 0s 6ms/step - loss: 1.3497 - accuracy: 0.2433
Epoch 3/200
19/19 [==============================] - 0s 5ms/step - loss: 1.3120 - accuracy: 0.3317
Epoch 4/200
19/19 [==============================] - 0s 5ms/step - loss: 1.2539 - accuracy: 0.4150
Epoch 5/200
19/19 [==============================] - 0s 5ms/step - loss: 1.1695 - accuracy: 0.4983
Epoch 6/200
19/19 [==============================] - 0s 7ms/step - loss: 1.0713 - accuracy: 0.5333
Epoch 7/200
19/19 [==============================] - 0s 5ms/step - loss: 0.9837 - accuracy: 0.5467
Epoch 8/200
19/19 [==============================] - 0s 5ms/step - loss: 0.9103 - accuracy: 0.5700
Epoch 9/200
19/19 [==============================] - 0s 7ms/step - loss: 0.8714 - accuracy: 0.6267
Epoch 10/200
19/19 [==============================] - 0s 6ms/step - loss: 0.8401 - accuracy: 0.6483
Epoch 11/200
19/19 [==============================] - 0s 5ms/step - loss: 0.8240 - accuracy: 0.6500
Epoch 12/2

In [21]:
# Evaluate your model accuracy on the test data
accuracy = model.evaluate(coord_test, competitors_test)[1]

7/7 [==============================] - 0s 4ms/step - loss: 0.6598 - accuracy: 0.8000


In [22]:
# Print accuracy
print('Accuracy:', accuracy)

Accuracy: 0.800000011920929


## Softmax predictions

In [27]:
import numpy as np

In [28]:
coords_small_test = pd.DataFrame({
    'xCoord':[0.209048, 0.082103, 0.198165, -0.348660, 0.214726],
    'yCoord':[-0.077398, -0.721407, -0.674646, 0.035086, 0.183894]
})

In [29]:
competitors_small_test = np.array([[0., 0., 1., 0.], [0., 0., 0., 1.],
                                   [0., 0., 0., 1.], [1., 0., 0., 0.],
                                   [0., 0., 1., 0.]])

In [30]:
# Predict on coords_small_test
preds = model.predict(coords_small_test)

In [31]:
# Print preds vs true values
print("{:45} | {}".format('Raw Model Predictions','True labels'))
for i,pred in enumerate(preds):
  print("{} | {}".format(pred,competitors_small_test[i]))

Raw Model Predictions                         | True labels
[0.36757675 0.01388432 0.60869336 0.00984556] | [0. 0. 1. 0.]
[0.07597809 0.00095835 0.03287261 0.8901909 ] | [0. 0. 0. 1.]
[0.2729116  0.00389942 0.15097886 0.5722101 ] | [0. 0. 0. 1.]
[0.9237413  0.03597204 0.03818503 0.00210172] | [1. 0. 0. 0.]
[0.28337267 0.01434216 0.6930492  0.00923595] | [0. 0. 1. 0.]


In [32]:
# Extract the position of highest probability from each pred vector
preds_chosen = [np.argmax(pred) for pred in preds]

In [34]:
# Print preds vs true values
print("{:10} | {}".format('Rounded Model Predictions','True labels'))
for i,pred in enumerate(preds_chosen):
    print("{:25} | {}".format(pred,competitors_small_test[i]))

Rounded Model Predictions | True labels
                        2 | [0. 0. 1. 0.]
                        3 | [0. 0. 0. 1.]
                        3 | [0. 0. 0. 1.]
                        0 | [1. 0. 0. 0.]
                        2 | [0. 0. 1. 0.]
